In [1]:
from sklearn.datasets import load_digits
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split
import keras
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.regularizers import l2
from keras import regularizers
from sklearn.model_selection import StratifiedShuffleSplit

Using TensorFlow backend.


In [0]:
dig = load_digits()

In [0]:
X = dig['data']
y = dig['target']
one_hot_labels = keras.utils.to_categorical(y, num_classes=10)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, one_hot_labels, random_state=0)

In [0]:
model = Sequential()
model.add(Dense(128, input_dim=64))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
model.fit(X_train,y_train,validation_split=0.2, epochs=10)

Train on 1077 samples, validate on 270 samples
Epoch 1/10
1077/1077 [==============================] - 0s 134us/step - loss: 2.7132 - accuracy: 0.3993 - val_loss: 0.6930 - val_accuracy: 0.7444
Epoch 2/10
1077/1077 [==============================] - 0s 50us/step - loss: 0.4126 - accuracy: 0.8728 - val_loss: 0.3099 - val_accuracy: 0.9074
Epoch 3/10
1077/1077 [==============================] - 0s 50us/step - loss: 0.2031 - accuracy: 0.9378 - val_loss: 0.1915 - val_accuracy: 0.9407
Epoch 4/10
1077/1077 [==============================] - 0s 48us/step - loss: 0.1300 - accuracy: 0.9638 - val_loss: 0.1311 - val_accuracy: 0.9667
Epoch 5/10
1077/1077 [==============================] - 0s 48us/step - loss: 0.0898 - accuracy: 0.9759 - val_loss: 0.1182 - val_accuracy: 0.9704
Epoch 6/10
1077/1077 [==============================] - 0s 49us/step - loss: 0.0722 - accuracy: 0.9842 - val_loss: 0.1007 - val_accuracy: 0.9741
Epoch 7/10
1077/1077 [==============================] - 0s 48us/step - loss: 0.046

In [0]:
def create_model(neurons_1 = 16, neurons_2 = 16, l2_penalty = 0.01):
  model = Sequential()
  model.add(Dense(neurons_1, input_dim=64, kernel_regularizer= l2(l2_penalty)))
  model.add(Activation('relu'))
  model.add(Dense(neurons_2, kernel_regularizer=l2(l2_penalty)))
  model.add(Activation('relu'))
  model.add(Dense(10))
  model.add(Activation('softmax'))
  model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  return model

In [0]:
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32)

In [0]:
ss = StratifiedShuffleSplit(n_splits=2, test_size=0.25, random_state=0)

In [15]:
neurons_1 = [64, 128, 256]
neurons_2 = [64, 128, 256]
l_2 = [0.01, 0.1, 1]
param_grid = dict(neurons_1 = neurons_1, neurons_2 = neurons_2, l2_penalty = l_2)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=ss)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Epoch 1/10
1347/1347 [==============================] - 0s 106us/step - loss: 3.2591 - accuracy: 0.5672
Epoch 2/10
1347/1347 [==============================] - 0s 39us/step - loss: 1.5990 - accuracy: 0.9198
Epoch 3/10
1347/1347 [==============================] - 0s 47us/step - loss: 1.3163 - accuracy: 0.9629
Epoch 4/10
1347/1347 [==============================] - 0s 44us/step - loss: 1.1638 - accuracy: 0.9762
Epoch 5/10
1347/1347 [==============================] - 0s 46us/step - loss: 1.0617 - accuracy: 0.9822
Epoch 6/10
1347/1347 [==============================] - 0s 41us/step - loss: 0.9863 - accuracy: 0.9874
Epoch 7/10
1347/1347 [==============================] - 0s 44us/step - loss: 0.9089 - accuracy: 0.9918
Epoch 8/10
1347/1347 [==============================] - 0s 44us/step - loss: 0.8567 - accuracy: 0.9926
Epoch 9/10
1347/1347 [==============================] - 0s 41us/step - loss: 0.7967 - accuracy: 0.9955
Epoch 10/10
1347/1347 [==============================] - 0s 52us/step - 

In [18]:
res = grid.score(X_test,y_test)
print("Accuracy on test data = ", round(res,4)*100,"%")

450/450 [==============================] - 0s 37us/step
Accuracy on test data =  96.89 %


We see that the Grid Search gives us the best parameters when first hidden layers has 128 units, second hidden layer has 64 units and the l2 penalty is 0.01. This model gives a 96.89% accuracy on the test data.